In [20]:
prompts = [
    "An image of a pntlng animation character swimming at the beach",
    "An image of a pntlng animation playing in snow",
    "An image of a pntlng animation playing guitar",
    "An image of a pntlng animation character going to school",
    "An image of a pntlng animation character on a roller coaster"
]
from diffusers.utils import load_image


In [21]:
from diffusers import DiffusionPipeline
import torch

from diffusers import StableDiffusionPipeline
import torch

model_id = "/home/jrajan/dreambooth/models/pointling"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")



Loading pipeline components...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.17it/s]


In [22]:
images = pipe(prompts, num_samples=30, num_inference_steps=50, guidance_scale=7.5, width=512, height=512,  output_type="np").images
images.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.94it/s]


(5, 512, 512, 3)

In [23]:
from torchmetrics.functional.multimodal import clip_score
from functools import partial

clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(images, prompts):
    images_int = (images * 255).astype("uint8")
    clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(clip_score), 4)

sd_clip_score = calculate_clip_score(images, prompts)
print(f"CLIP score: {sd_clip_score}")

CLIP score: 31.2293
